# 1 RoPE（Rotary Position Embedding，旋转位置编码）

根据embed_dim，维度的不同位置，具有不同的频率值：
$$
freq = \frac{1}{base^{2i / dim}}
$$

根据seq_len，序列中不同位置，具备不同的系数m：
$$
m = i, i \in [0, seq\_len - 1]
$$

由此计算出每一个token的每一个维度位置的旋转角度：
$$
angles = m * freq
$$



In [36]:
import torch


class RoPE:
    def __init__(self, base: int = 10000):
        self.base = base

    def apply_rope(self, x):
        batch_size, seq_len, embed_dim = x.shape

        # 根据embed_dim，生成不同维度的频率
        freqs = self.base ** (-torch.arange(0, embed_dim, 2) / embed_dim)
        # 根据max_len，生成位置索引，扩展一个embed_dim维度
        m = torch.arange(seq_len).unsqueeze(1)
        # 每个位置的旋转角度
        angles = freqs * m

        # 分别计算sin和cos
        sin = torch.sin(angles)
        cos = torch.cos(angles)

        # 拆分为偶数、奇数序列，并且两两为一个组
        x1, x2 = x[..., 0::2], x[..., 1::2]

        rotated1 = x1 * cos - x2 * sin
        rotated2 = x1 * sin + x2 * cos

        return torch.stack([rotated1, rotated2], dim=-1).flatten(-2)


In [37]:
rope = RoPE()
rope.apply_rope(torch.randn(1, 2, 16))

tensor([[[ 0.8016,  0.9471,  0.4519,  0.2748,  0.9806,  0.4978,  1.4450,
           0.4957,  0.2457,  0.3452, -0.1276, -0.5308, -1.4016, -0.4081,
           0.5138,  1.9119],
         [ 0.7372, -1.0070, -0.3985, -1.0715,  0.7012, -0.0459,  0.0783,
          -0.0165,  1.4683, -1.7392, -0.8366,  0.3128,  1.7088,  0.2354,
           0.1994, -0.7368]]])